Conciliación carrier Internacional

En este notebook se ha desarrollado una herramienta para la concialicación de la propuesta de carrier internacional. Además, da información en ese mes de análisis de los beneficios/pérdidas que ha supuesto el carrier por país.

Consideraciones:
    .Se nutre de:
        1. Crudo de OCS cuando b_tipollamada es RoamingVoiceMOC,RoamingVoiceMTC e InternationalVoice.
        2. Propuesta facturación del proveedor.
        3. AZ del proveedor.
   
   .Categoriza OCS con el AZ del proveedor.

------------------------------------------------------------------------------------------

Este proceso realiza dos subprocesos para poder categorizar los eventos, aunque ambos utilizan la misma lógica.
La lógica es:

    DictF = {}

    for k in called:
    k=str(k)
    i=0
    okei = False
    for i in range(len(k)):
        cc = left(k,len(k)-i)
        if cc in DictCode:
            DictF[k]= cc
            okei = True 
            break
    if not okei: 
        print("Error {}".format(k))
---------------------------------------------------------------------------------------------        
Por un lado se realiza cuando b_tipollamada es InternationalVoice y RoamingVoiceMOC. Y por otro, RoamingVoiceMTC. En el primer punto se busca la máxima coincidencia por b_numerollamado con el AREA Code de TESA y en el segundo punto se utiliza el Country Code Location. Una vez realizado los dos subprocesos se realiza un join y vuelven a participar de un proceso común.
        

# Librerías

In [ ]:
import sys
sys.path.append('..')

from modulodirectorio import*
from Utils import*
from bbdd_conect import*


import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
from bokeh.models import HoverTool, BoxSelectTool
from bokeh.io import output_notebook
from bokeh.plotting import figure, show, output_file
from bokeh.models import FactorRange
from bokeh.models import Title
from pivottablejs import pivot_ui
from IPython.display import HTML
output_notebook()
    
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

Inputs:

In [ ]:

DictOp = pd.read_excel(get_path('dict_az_country'))
DictTar = pd.read_excel(get_path('dict_ocs_country'))

mes_revision = 202306
query = query_database('query_carrier_internacional', mes=mes_revision) 
RawCDR = query


path = get_path('az_carrier')
filename = input("Ingresar el nombre del archivo del AZ nuevo: ") + ".csv" 
fullpath = os.path.join(path, filename)
    
    
AZ1= pd.read_csv(fullpath)

path = get_path('prop_dashboard')
filename = input("Ingresar el nombre del archivo de la propuesta pasada: ") + ".csv" 
fullpath = os.path.join(path, filename)
    
    
Pro= pd.read_csv(fullpath)


Normalización de datos:

In [ ]:
Pro = Pro.rename(columns={"DESCRIPCION": "Destination"})
#Realizamos un merge para generar una columna nueva que permita traducir la columna Destcountry en un pais, esto es la categorización del tarificador.
RawCDR=pd.merge(RawCDR, DictTar,  how='left', left_on=['b_paisdestino'], right_on = ['DestCountry'])
#Unificador de AZ, la lógica consiste en coger la última fecha de aplicación por operador
#realizamos un concat de todos los AZ recibidos en un mes
AZ = pd.concat([AZ1, AZ2], ignore_index=True)
AZ = AZ[['Rate', 'Effective Date', 'Area Code', 'Destination']]
#Ordenamos el dataframe por fecha de aplicación
AZ = AZ.sort_values('Effective Date',ascending=False)
#Agrupamos por MSISDN y cogemos el primer valor, para relacionar el MSISDN con su estado más actual respecto al operador.
AZ = AZ.groupby('Area Code').first().reset_index()
AZRate = AZ[['Rate', 'Area Code', 'Destination']]
AZRateM = AZ[['Area Code']]
AZRateM = AZRateM['Area Code'].tolist()
AZRateM = [int(a) for a in AZRateM]
AZRateM = [str(a) for a in AZRateM]
AZRate['Area Code'] = AZRateM
AZRate= AZRate[['Rate','Area Code', 'Destination']]
#Modificamos a string en pandas
AZDest=AZRate[['Rate','Area Code', 'Destination']]#Incluimos el pais al que pertenece el operador
Pro = Pro.merge(DictOp, on="Destination", how='left')
#Renombramos el valor de Origzone en llamada internacional para normalizar los datos
RawCDR['b_bandacosteorigen'] = RawCDR['b_bandacosteorigen'].replace(['NACIONAL'],['Zona 1'])
Code = AZ['Area Code'].astype(str)
RawCDR['duracion'] = RawCDR['b_duracion']/60
RawCDR['Fecha']  = pd.to_datetime(RawCDR['b_fechainicio'])
RawCDR['Fecha'] = RawCDR['Fecha'].dt.date

Proceso común a ambos:

In [ ]:
#Elemento intrínseco a los dos procesos

DictCode = {}
for i in Code:
    DictCode[i]= '' 

Proceso 1: InternacionalVoice and RoamingVoiceMOC

In [ ]:
RawCDRM = RawCDR[(RawCDR['b_tipollamada'] == "InternationalVoice") | (RawCDR['b_tipollamada'] == "RoamingVoiceMOC")] #Me quito RoamingVoiceMTC

RawCDRM.drop(RawCDRM.loc[RawCDRM['b_paisdestino']=='ESPAÑA'].index, inplace=True)

called = RawCDRM['b_numerollamado'].fillna(0)
called = RawCDRM['b_numerollamado'].tolist()

#quitar duplicados en una lista
called = list(set(called))
called = [int(a) for a in called] 


#Coincidencias InternationalVoice y RoamingVoiceMOC
DictF = {}

for k in called:
    k=str(k)
    i=0
    okei = False
    for i in range(len(k)):
        cc = left(k,len(k)-i)
        if cc in DictCode:
            DictF[k]= cc
            okei = True 
            break
    if not okei: 
        print("Error {}".format(k))
              
#Convertimos el diccionario a pandas       
NumB_Code = pd.DataFrame.from_dict(DictF, orient='index').reset_index()
NumB_Code.columns = ['b_numerollamado', 'AreaCode']
NumB_Code = [str(a) for a in NumB_Code]
NumB_Code = pd.DataFrame.from_dict(DictF, orient='index').reset_index()
NumB_Code.columns = ['b_numerollamado', 'Area Code']

#Unimos con un merge para relacionar MSISDN con los operadores
NumB_Code = NumB_Code.merge(AZRate, on="Area Code", how='left')
NumBfix = RawCDRM['b_numerollamado'].tolist() #lo queremos con duplicados

#Para poder  hacer el merge más adelante los datos tienen que pasar por los mismo estados antes de llegar al formato string
NumBfix = [int(a) for a in NumBfix]
NumBfix = [str(a) for a in NumBfix]
RawCDRMFinal = RawCDRM[['b_fechainicio','b_uniquerecordidentifier','b_numerollamante','b_tipollamada','Fecha','duracion', 'b_numerollamado','Pais','b_coste','b_volumen','b_bandacosteorigen','b_bandacostedestino','b_paisorigen','b_paisdestino','b_offername_1']]

#añado la columna NumB sin ser float
RawCDRMFinal['b_numerollamado'] = NumBfix

#Relacionamos la volumetría del tarificador con el dataframe con los operadores
RawCDRMFinal = RawCDRMFinal.merge(NumB_Code, on="b_numerollamado", how='left')

#Generamos la columna con el coste del evento
RawCDRMFinal['PriceCall']= RawCDRMFinal['duracion'] * RawCDRMFinal['Rate']

#Traduzco el nombre del operador para incluir mi categorización de pais.
RawCDRMFinal = RawCDRMFinal.merge(DictOp, on="Destination", how='left')

Proceso 2: RoamingVoiceMTC

In [ ]:
#RoamingVoiceMTC
RawCDRMTC = RawCDR[(RawCDR['b_tipollamada'] == "RoamingVoiceMTC")]
RawCDRMTC.drop(RawCDRMTC.loc[RawCDRMTC['b_paisdestino']=='ESPAÑA'].index, inplace=True)

cclocation=RawCDRMTC['cclocation']
cclocation = [int(a) for a in cclocation]

#Coincidencias entre MTC y AreaCode del AZ
#Aquí creo un diccionario y recorro cclocation por el diccionario de Códigos quitandole dígitos hasta encontrar la coincidencia de mayor longitud
DictFMTC = {}

for k in cclocation:
    k=str(k)
    i=0
    okei = False
    for i in range(len(k)):
        cc = left(k,len(k)-i)
        if cc in DictCode:
            DictFMTC[k]= cc
            okei = True 
            break
    if not okei: 
        print("Error {}".format(k))
        
        
#Convierto el diccionario en un dataframe
CC_Code = pd.DataFrame.from_dict(DictFMTC, orient='index').reset_index()
CC_Code.columns = ['b_numerollamado', 'Area Code']

#Convierto las variables a string
CC_Code = [str(a) for a in CC_Code]
CC_Code = pd.DataFrame.from_dict(DictFMTC, orient='index').reset_index()
CC_Code.columns = ['b_numerollamado', 'Area Code']

#Lo cruzo con el AZ para sacar el operador
CC_Code = CC_Code.merge(AZRate, on="Area Code", how='left')
CCfix = [int(a) for a in cclocation]
CCfix = [str(a) for a in cclocation]
RawCDRMTCFinal = RawCDRMTC[['b_fechainicio','b_uniquerecordidentifier','b_numerollamante','b_tipollamada','Fecha','duracion', 'b_numerollamado','Pais','b_coste','b_volumen','b_bandacosteorigen','b_bandacostedestino','b_paisorigen','b_paisdestino','b_offername_1']]

#Añadimos la columna NumB sin ser float
RawCDRMTCFinal['b_numerollamado'] = CCfix
RawCDRMTCFinal = RawCDRMTCFinal.merge(CC_Code, on="b_numerollamado", how='left')

#Genero una columna nueva de coste total por evento
RawCDRMTCFinal['PriceCall']= RawCDRMTCFinal['duracion'] * RawCDRMTCFinal['Rate']
RawCDRMTCFinal = RawCDRMTCFinal.merge(DictOp, on="Destination", how='left')

Concatenamos los dos dataframe generados en el proceso 1 y 2

In [ ]:
ResumenFinal = pd.concat([RawCDRMTCFinal, RawCDRMFinal], ignore_index=True)

#Tradudimos la columna de los operadores para identificar el pais bajo la categorización que hacemos bajo este proceso y no, con la del tarificador..

ResumenFinal = ResumenFinal[ResumenFinal['duracion'] != 0]

In [ ]:
#Agrupamos por la categorización creada
DisCountryprocessed = ResumenFinal.groupby(by='Paisprocessed').agg({'b_coste': lambda x: sum(x),
                                                                    'duracion': 'sum',
                                                                    'PriceCall': 'sum',
                                                                    'b_uniquerecordidentifier':'nunique'}).reset_index()

DisOpeprocessed = ResumenFinal.groupby(by='Destination').agg({'b_coste': lambda x: sum(x),
                                                              'duracion': 'sum',
                                                              'PriceCall': 'sum',
                                                              'b_uniquerecordidentifier':'nunique'}).reset_index()

#Agrupamos el dataframe de la propuesta facturada
PropOp = Pro.groupby(by='Destination').sum().reset_index()
PropOp = PropOp[['Destination','DURACION','IMPORTES','LLAMADAS']]
Pro = Pro.groupby(by='Paisprocessed').sum().reset_index()
Pro = Pro[['Paisprocessed','DURACION','IMPORTES','LLAMADAS']]

#Hacemos un join por pais para incluir la volumetría por pais y el precio
DisCountryprocessed = DisCountryprocessed.merge(Pro, on="Paisprocessed", how='outer')
DisCountryprocessed = DisCountryprocessed.fillna(0)

DisOpeprocessed = DisOpeprocessed.merge(PropOp, on="Destination", how='outer')

#Creamos las columnas con el porcentaje de error por minuto e importe, para ver si la discrepancias está en volumetría o en coste.
DisCountryprocessed['DifPMIN']= (abs((DisCountryprocessed['duracion'] - DisCountryprocessed['DURACION'])/DisCountryprocessed['DURACION'])*100).round(2)
DisCountryprocessed['DifPIMPORTE']= (abs((DisCountryprocessed['PriceCall'] - DisCountryprocessed['IMPORTES'])/DisCountryprocessed['IMPORTES'])*100).round(2)

#Una vez hemos agrupado para sacer el beneficio por pais restamos el importe agregado de lo que le repercutimos al cliente menos loq ue nos repercuten a nuestra compañia por las conexiones.
DisCountryprocessed['DifBen']= DisCountryprocessed['IMPORTES'] - DisCountryprocessed['PriceCall']

#Ordenamos por los paises que tienen más volumetría, porque ordenar por discrepancia no tiene sentido porque los paises con muy poca volumetría te van a salir un error muy alto.
DisCountryprocessed = DisCountryprocessed.sort_values('duracion',ascending=False)

#calculamos la diferencia de minutos
DisCountryprocessed['DifMIN']= abs((DisCountryprocessed['duracion'] - DisCountryprocessed['DURACION'])).round(2)
DisOpeprocessed['DifMIN']= abs((DisOpeprocessed['duracion'] - DisOpeprocessed['DURACION'])).round(2)

#Diferncia de importe
DisOpeprocessed['DifBen']= abs((DisOpeprocessed['IMPORTES'] - DisOpeprocessed['PriceCall'])).round(2)

#Precio unitario llamada
DisOpeprocessed['Importe unitario TESA']= abs((DisOpeprocessed['IMPORTES']/DisOpeprocessed['DURACION'])).round(4)
DisOpeprocessed['Importe unitario AZ']= abs((DisOpeprocessed['PriceCall']/DisOpeprocessed['duracion'])).round(4)

#Beneficio
DisCountryprocessed['Beneficio/perdidas']= (DisCountryprocessed['b_coste'] - DisCountryprocessed['PriceCall']).round(2)
DisOpeprocessed['Beneficio/perdidas']= (DisOpeprocessed['b_coste'] - DisOpeprocessed['PriceCall']).round(2)

#DisOpeprocessed[['minutos OCS','b_coste','Mediacion coste TESA','minutos TESA','Importe Facturado TESA']] =  DisOpeprocessed[['minutos OCS','b_coste','Mediacion coste TESA','minutos TESA','Importe Facturado TESA']].round(2)
#DisCountryprocessed[['minutos OCS','b_coste','Mediacion coste TESA','minutos TESA','Importe Facturado TESA','Diferencia Precios']] =  DisCountryprocessed[['minutos OCS','b_coste','Mediacion coste TESA','minutos TESA','Importe Facturado TESA','Diferencia Precios']].round(2)

# Output .xlsx:

In [ ]:
ResumenFinal.to_csv ('Detalle Carrier internacional procesado.csv', sep=',', index=False)
#DisCountryprocessed.to_excel('Conciliación carrier internacional procesado.xlsx', sheet_name='sheet1', index=True)

Consolidación carrier con la categorización creada

In [ ]:
Graf1 = DisCountryprocessed.head(10)

fig, ax = plt.subplots()
plt.title('Minutos reconocidos por país')
ax.plot(Graf1['Paisprocessed'], Graf1['duracion'], marker = '^', label = 'Tarificador Procesado')
ax.plot(Graf1['Paisprocessed'], Graf1['DURACION'], marker = 'o', label = 'Propuesta Carrier')

fig.set_size_inches(15, 6)

ax.set_xlabel("País")
ax.set_ylabel("Minutos")

ax.legend(loc = 'upper right')

plt.show()

In [ ]:
filepath1=get_path('analisis_carrier_output')

DisCountryprocessed = DisCountryprocessed.rename(columns={"Paisprocessed": "Pais Destino Reclasificado",
                                                          "duracion": "minutos OCS",
                                                          "DURACION": "minutos TESA",
                                                          "PriceCall": "Mediacion coste TESA",
                                                          "IMPORTES": "Importe Facturado TESA",
                                                          "DifPIMPORTE": "Diferencia Porcentual Precios",
                                                          "DifPMIN": "Diferencia Porcentual Minutos",
                                                          "DifMIN": "Diferencia Minutos",
                                                          "DifBen": "Diferencia Precios",
                                                          "b_uniquerecordidentifier": "Nº llamadas OCS" })


DisOpeprocessed = DisOpeprocessed.rename(columns={"Paisprocessed": "Pais Destino Reclasificado",
                                                          "duracion": "minutos OCS",
                                                          "DURACION": "minutos TESA",
                                                          "PriceCall": "Mediacion coste TESA",
                                                          "IMPORTES": "Importe Facturado TESA",
                                                          "DifPIMPORTE": "Diferencia Porcentual Precios",
                                                          "DifPMIN": "Diferencia Porcentual Minutos",
                                                          "DifMIN": "Diferencia Minutos",
                                                          "DifBen": "Diferencia Precios",
                                                          "b_uniquerecordidentifier": "Nº llamadas OCS" })



#DisOpeprocessed[['minutos OCS','b_coste','Mediacion coste TESA','minutos TESA','Importe Facturado TESA']] =  DisOpeprocessed[['minutos OCS','b_coste','Mediacion coste TESA','minutos TESA','Importe Facturado TESA']].round(2)
#DisCountryprocessed[['minutos OCS','b_coste','Mediacion coste TESA','minutos TESA','Importe Facturado TESA','Diferencia Precios']] =  DisCountryprocessed[['minutos OCS','b_coste','Mediacion coste TESA','minutos TESA','Importe Facturado TESA','Diferencia Precios']].round(2)


pivot_ui(DisCountryprocessed,
    outfile_path=filepath1/"Conciliación Carrier paises.html",
    rendererName= "Table",
    rowOrder= "value_z_to_a", 
    colOrder= "value_z_to_a",
    rendererOptions= {
        "c3": { "data": {"colors": {
            "Liberal": '#dc3912', "Conservative": '#3366cc', "NDP": '#ff9900',
            "Green":'#109618', 'Bloc Quebecois': '#A05EB5'
        }}}
    }
)

# Definir los colores para el degradado
color1 = "#A05EB5"  # Color inicial
color2 = "#2A0A29"  # Color final

# Ruta de la imagen a añadir
ruta_imagen = get_path('image_html')

# Abrir el archivo HTML generado
with open(filepath1/"Conciliación Carrier paises.html", "r") as file:
    html_content = file.read()

# Modificar el código HTML para agregar el degradado de color al fondo y la imagen en el margen superior
modified_html_content = html_content.replace(
    "<head>",
    f"<head><style>body {{ background: linear-gradient(to bottom, {color1}, {color2}); }}</style>"
    f"<style>#image-container {{ position: absolute; top: 10px; left: 80%; transform: translateX(-50%); width: 25px; height: 25px; opacity: 0.7; }}</style>"
    f"</head><body><div id='image-container'><img src='{ruta_imagen}'></div>"
)

# Escribir el contenido modificado en el archivo HTMLs
with open(filepath1/"Conciliación Carrier paises.html", "w") as file:
    file.write(modified_html_content)


In [ ]:
pivot_ui(DisOpeprocessed,
    outfile_path=filepath1/"Conciliación Carrier operadores.html",
    rendererName= "Table",
    rowOrder= "value_z_to_a", 
    colOrder= "value_z_to_a",
    rendererOptions= {
        "c3": { "data": {"colors": {
            "Liberal": '#dc3912', "Conservative": '#3366cc', "NDP": '#ff9900',
            "Green":'#109618', 'Bloc Quebecois': '#990099'
        }}}
    }
)


# Ruta de la imagen a añadir
ruta_imagen = get_path('image_html')

# Abrir el archivo HTML generado
with open(filepath1/"Conciliación Carrier operadores.html", "r") as file:
    html_content = file.read()

# Modificar el código HTML para agregar el degradado de color al fondo y la imagen en el margen superior
modified_html_content = html_content.replace(
    "<head>",
    f"<head><style>body {{ background: linear-gradient(to bottom, {color1}, {color2}); }}</style>"
    f"<style>#image-container {{ position: absolute; top: 10px; left: 80%; transform: translateX(-50%); width: 25px; height: 25px; opacity: 0.7; }}</style>"
    f"</head><body><div id='image-container'><img src='{ruta_imagen}'></div>"
)

# Escribir el contenido modificado en el archivo HTML
with open(filepath1/"Conciliación Carrier operadores.html", "w") as file:
    file.write(modified_html_content)